In [224]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
import shutil
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
from sklearn.preprocessing import LabelBinarizer
import pickle
import cv2

In [225]:
""" 
/training_data_all
        --->Unknown
        --->Absent
        --->Present
"""

' \n/training_data_all\n        --->Unknown\n        --->Absent\n        --->Present\n'

In [226]:
def noise(data):
    noise_amp = 0.05*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

# def pitch(data, sampling_rate, pitch_factor=0.8):
#     return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)

def higher_speed(data):
    return librosa.effects.time_stretch(data, rate=1.2)

def lower_speed(data):
    return librosa.effects.time_stretch(data, rate=0.8)

In [229]:
def get_label(label):
    # print(folders)
    y_data.append(label)

def get_data(name_files,audio,time_limit,sr):

    segment_length = time_limit * sr
    num_segments = (len(audio) // segment_length)
    for i in range(num_segments):
        start_sample = i * segment_length
        end_sample = (i + 1) * segment_length
        segment = audio[start_sample:end_sample]
        create_data(segment,n_fft,n_mels,hop_length)

        label = name_files.split('/')[1]
        get_label(label)
def create_data (audio,n_fft,n_mels,hop_length):
    mel_spectrogram = librosa.feature.melspectrogram(y=audio,n_fft= n_fft ,n_mels=n_mels,hop_length=hop_length)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram ,ref=np.max)
    w,h= mel_spectrogram_db.shape
    mel_spectrogram_db = mel_spectrogram_db.reshape([w,h,1])
    x_data.append(mel_spectrogram_db)

In [234]:
time_limit = 2
sr = 4000
n_fft = 512
n_mels= 64
hop_length = 126

In [235]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Absent"
for file in os.listdir(name_folders):
    if(dem<=800):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data(name_files,audio,time_limit = time_limit,sr = sr)
    dem = dem +1
x_absent_spectrogram = x_data
y_absent_spectrogram = y_data
np.array(x_absent_spectrogram).shape

(8954, 64, 64, 1)

In [236]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Present"
for file in os.listdir(name_folders):
    if(True):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data(name_files,audio,time_limit = time_limit,sr = sr)
    dem = dem +1
x_present_spectrogram = x_data
y_present_spectrogram = y_data
np.array(x_present_spectrogram).shape

(6548, 64, 64, 1)

In [237]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Unknown"
for file in os.listdir(name_folders):
    if(True):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data(name_files,audio,time_limit = time_limit,sr = sr)
    dem = dem +1
x_unknown_spectrogram1 = x_data
y_unknown_spectrogram1 = y_data
np.array(x_unknown_spectrogram1).shape

(1662, 64, 64, 1)

In [238]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Unknown"
for file in os.listdir(name_folders):
    if(True):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data(name_files,noise(audio),time_limit = time_limit,sr = sr)
    dem = dem +1
x_unknown_spectrogram2 = x_data
y_unknown_spectrogram2 = y_data
np.array(x_unknown_spectrogram2).shape

(1662, 64, 64, 1)

In [239]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Unknown"
for file in os.listdir(name_folders):
    if(True):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data(name_files,higher_speed(audio),time_limit = time_limit,sr = sr)
    dem = dem +1
x_unknown_spectrogram3 = x_data
y_unknown_spectrogram3 = y_data
np.array(x_unknown_spectrogram3).shape

(1381, 64, 64, 1)

In [240]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Unknown"
for file in os.listdir(name_folders):
    if(True):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data(name_files,lower_speed(audio),time_limit = time_limit,sr = sr)
    dem = dem +1
x_unknown_spectrogram4 = x_data
y_unknown_spectrogram4 = y_data
np.array(x_unknown_spectrogram4).shape

(2110, 64, 64, 1)

In [241]:
x_data_spectrogram = np.concatenate((x_absent_spectrogram,
                                     x_present_spectrogram,
                                     x_unknown_spectrogram1,
                                     x_unknown_spectrogram2,
                                     x_unknown_spectrogram3,
                                     x_unknown_spectrogram4),axis=0)
y_data_spectrogram = np.concatenate((y_absent_spectrogram,
                                     y_present_spectrogram,
                                     y_unknown_spectrogram1,
                                     y_unknown_spectrogram2,
                                     y_unknown_spectrogram3,
                                     y_unknown_spectrogram4),axis= 0)

In [269]:
x_data_spectrogram.shape

(22071, 64, 64, 1)

In [242]:

def get_data_mfccs(name_files,audio,time_limit,sr):
    segment_length = time_limit * sr
    num_segments = (len(audio) // segment_length)
    for i in range(num_segments):
        start_sample = i * segment_length
        end_sample = (i + 1) * segment_length
        segment = audio[start_sample:end_sample]

        create_data_mfccs(segment,n_mfcc)

        label = name_files.split('/')[1]
        get_label(label)
def create_data_mfccs (audio,n_mfcc):
    mel_mfcc = librosa.feature.mfcc(y=audio, sr=sr,n_mfcc=n_mfcc)
    mel_mfcc= np.resize(mel_mfcc,(64,64))
    mel_mfcc= np.reshape(mel_mfcc,(64,64,1))
    x_data.append(mel_mfcc)

In [243]:

n_mfcc = 64

In [244]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Absent"
for file in os.listdir(name_folders):
    if(800<dem<=1600):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data_mfccs(name_files,audio,time_limit = time_limit,sr = sr)
    dem = dem +1
x_absent_mfccs = x_data
y_absent_mfccs = y_data
np.array(x_absent_mfccs).shape

(8708, 64, 64, 1)

In [245]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Present"
for file in os.listdir(name_folders):
    if(True):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data_mfccs(name_files,audio,time_limit = time_limit,sr = sr)
    dem = dem +1
x_present_mfccs = x_data
y_present_mfccs = y_data
np.array(x_present_mfccs).shape

(6548, 64, 64, 1)

In [246]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Unknown"
for file in os.listdir(name_folders):
    if(True):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data_mfccs(name_files,audio,time_limit = time_limit,sr = sr)
    dem = dem +1
x_unknown_mfccs1 = x_data
y_unknown_mfccs1 = y_data
np.array(x_unknown_mfccs1).shape

(1662, 64, 64, 1)

In [247]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Unknown"
for file in os.listdir(name_folders):
    if(True):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data_mfccs(name_files,noise(audio),time_limit = time_limit,sr = sr)
    dem = dem +1
x_unknown_mfccs2 = x_data
y_unknown_mfccs2 = y_data
np.array(x_unknown_mfccs2).shape

(1662, 64, 64, 1)

In [248]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Unknown"
for file in os.listdir(name_folders):
    if(True):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data_mfccs(name_files,higher_speed(audio),time_limit = time_limit,sr = sr)
    dem = dem +1
x_unknown_mfccs3 = x_data
y_unknown_mfccs3 = y_data
np.array(x_unknown_mfccs3).shape

(1381, 64, 64, 1)

In [249]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Unknown"
for file in os.listdir(name_folders):
    if(True):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data_mfccs(name_files,lower_speed(audio),time_limit = time_limit,sr = sr)
    dem = dem +1
x_unknown_mfccs4 = x_data
y_unknown_mfccs4 = y_data
np.array(x_unknown_mfccs4).shape

(2110, 64, 64, 1)

In [251]:
x_data_mfccs = np.concatenate((x_absent_mfccs,
                                     x_present_mfccs,
                                     x_unknown_mfccs1,
                                     x_unknown_mfccs2,
                                     x_unknown_mfccs3,
                                     x_unknown_mfccs4),axis=0)
y_data_mfccs = np.concatenate((y_absent_mfccs,
                                     y_present_mfccs,
                                     y_unknown_mfccs1,
                                     y_unknown_mfccs2,
                                     y_unknown_mfccs3,
                                     y_unknown_mfccs4),axis= 0)

In [268]:
x_data_mfccs.shape

(22071, 64, 64, 1)

In [253]:
n_fft=512
hop_length=1250
sr=4000

In [254]:

def get_data_chroma(name_files,audio,time_limit,sr):

    segment_length = time_limit * sr
    num_segments = (len(audio) // segment_length)
    for i in range(num_segments):
        start_sample = i * segment_length
        end_sample = (i + 1) * segment_length
        segment = audio[start_sample:end_sample]

        create_data_chroma(segment,n_fft,hop_length)

        label = name_files.split('/')[1]
        get_label(label)
def create_data_chroma (audio,n_fft,hop_length):
    chromagram = librosa.feature.chroma_stft(y=audio,n_fft=n_fft,hop_length=hop_length)
    chromagram = cv2.resize(chromagram, (64,64))
    chromagram= np.resize(chromagram,(64,64,1))
    x_data.append(chromagram)

In [255]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Absent"
for file in os.listdir(name_folders):
    if(dem>1600):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data_chroma(name_files,audio,time_limit = time_limit,sr = sr)
    dem = dem +1 
x_absent_chroma= x_data
y_absent_chroma = y_data
np.array(x_absent_chroma).shape

/home/duyphuoc/Desktop/heart_sound_new/venv/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


(8686, 64, 64, 1)

In [261]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Present"
for file in os.listdir(name_folders):
    if(True):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data_chroma(name_files,audio,time_limit = time_limit,sr = sr)
    dem = dem +1
x_present_chroma = x_data
y_present_chroma = y_data
np.array(x_present_chroma).shape

/home/duyphuoc/Desktop/heart_sound_new/venv/lib/python3.10/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


(6548, 64, 64, 1)

In [262]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Unknown"
for file in os.listdir(name_folders):
    if(True):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data_chroma(name_files,audio,time_limit = time_limit,sr = sr)
    dem = dem +1
x_unknown_chroma1 = x_data
y_unknown_chroma1 = y_data
np.array(x_unknown_chroma1).shape

(1662, 64, 64, 1)

In [263]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Unknown"
for file in os.listdir(name_folders):
    if(True):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data_chroma(name_files,noise(audio),time_limit = time_limit,sr = sr)
    dem = dem +1
x_unknown_chroma2 = x_data
y_unknown_chroma2 = y_data
np.array(x_unknown_chroma2).shape

(1662, 64, 64, 1)

In [264]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Unknown"
for file in os.listdir(name_folders):
    if(True):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data_chroma(name_files,higher_speed(audio),time_limit = time_limit,sr = sr)
    dem = dem +1
x_unknown_chroma3 = x_data
y_unknown_chroma3 = y_data
np.array(x_unknown_chroma3).shape

(1381, 64, 64, 1)

In [265]:
x_data = []
y_data = []
dem = 0
name_folders = "training_data_all/Unknown"
for file in os.listdir(name_folders):
    if(True):
        name_files = os.path.join(name_folders,file)
        audio, sr = librosa.load(name_files, sr=sr)
        get_data_chroma(name_files,lower_speed(audio),time_limit = time_limit,sr = sr)
    dem = dem +1
x_unknown_chroma4 = x_data
y_unknown_chroma4 = y_data
np.array(x_unknown_chroma4).shape

(2110, 64, 64, 1)

In [266]:
x_data_chroma = np.concatenate((x_absent_chroma,
                                     x_present_chroma,
                                     x_unknown_chroma1,
                                     x_unknown_chroma2,
                                     x_unknown_chroma3,
                                     x_unknown_chroma4),axis=0)
y_data_chroma = np.concatenate((y_absent_chroma,
                                     y_present_chroma,
                                     y_unknown_chroma1,
                                     y_unknown_chroma2,
                                     y_unknown_chroma3,
                                     y_unknown_chroma4),axis= 0)

In [267]:
x_data_chroma.shape

(22049, 64, 64, 1)

In [295]:
x_data_all = []
x_data_all.append(x_data_spectrogram)
x_data_all.append(x_data_mfccs)
x_data_all.append(x_data_chroma)

y_data_all = []
y_data_all.append(y_data_spectrogram)
y_data_all.append(y_data_mfccs)
y_data_all.append(y_data_chroma)

In [303]:

file = open('data_all.csv','wb')
pickle.dump((x_data_all,y_data_all),file)
file.close()

: 